### Part 4. Langchain - Agent
- Objectives: Langchain 라이브러리를 활용하여 Agent기능 알아보기

In [2]:
import os
import time
from dotenv import load_dotenv
load_dotenv()

pinecone_api_key = os.environ.get('PINECONE_API_KEY')
openai_api_key = os.environ.get('OPENAI_API_KEY')

In [7]:
# 임베딩 모델 정의
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=openai_api_key
)

In [5]:
# 파인콘 인덱스 연결
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

index_name="quickstart"
index = pc.Index(index_name)

In [8]:
#Langchain에서 벡터스토어 연결성 맺기
from langchain.vectorstores import Pinecone

text_field = "text"

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

C:\Users\sckim\.conda\envs\vectordb\Lib\site-packages\langchain_community\vectorstores\pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [9]:
query = "지미 카터가 누구야?"

vectorstore.similarity_search(
    query,
    k=3
)

[Document(page_content='제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.', metadata={'chunk': 0.0, 'id': '5', 'source': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0', 'title': '지미 카터'}),
 Document(page_content='퇴임 이후 \n\n퇴임 이후 민간 자원을 적극 활용한 비영리 기구인 카터 재단을 설립한 뒤 민주주의 실현을 위해 제 3세계의 선거 감시 활동 및 기니 벌레에 의한 드라쿤쿠르스 질병 방재를 위해 힘썼다. 미국의 빈곤층 지원 활동, 사랑의 집짓기 운동, 국제 분쟁 중재 등의 활동도 했다.\n\n카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 선행하는 행위에 대해 깊은 유감을 표시 하며 미국의 군사적 활동에 강한 반대 입장을 보이고 있다.', metadata={'chunk': 6.0, 'id': '5', 'source': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0', 'title': '지미 카터'}),
 Document(page_content='지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통령 중 하나다. 인권 문제와 주한미군 철수 문제로 한때 한미 관계가 불편하기도 했다

In [10]:
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-4-turbo-preview',
    temperature=0.0
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5, # 참조할 지난 대화의 수
    return_messages=True
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [11]:
qa.invoke(query)

C:\Users\sckim\.conda\envs\vectordb\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'지미 카터는 민주당 출신 미국 39대 대통령(1977년 ~ 1981년)입니다. 그는 조지아주 플레인스 마을에서 태어났으며, 조지아 공과대학교를 졸업한 후 해군에서 전함, 원자력, 잠수함의 승무원으로 일했습니다. 해군 대위로 예편한 후에는 땅콩과 면화 농사를 지어 "땅콩 농부"라는 별명을 얻었습니다. 퇴임 후에는 카터 재단을 설립하여 민주주의 실현을 위한 제3세계 선거 감시 활동, 기니 벌레에 의한 드라쿤쿠르스 질병 방재, 미국의 빈곤층 지원, 사랑의 집짓기 운동, 국제 분쟁 중재 등 다양한 활동을 했습니다. 또한, 그는 미국의 군사적 행동에 대해 강한 반대 입장을 보였으며, 대한민국과의 관계에서도 인권 문제와 주한미군 철수 문제로 한때 불편한 관계를 가졌으나, 방한을 통해 관계가 다소 회복되었습니다.'

In [12]:
# 에이전트 없이 질문해보기
qa.invoke('오늘 야식 메뉴좀 추천해줘')

{'query': '오늘 야식 메뉴좀 추천해줘',
 'result': '야식 메뉴로는 다양한 옵션이 있습니다. 여러분의 취향이나 상황에 따라 선택할 수 있는 몇 가지를 추천해 드리겠습니다:\n\n1. **치킨**: 언제 어디서나 사랑받는 야식 메뉴입니다. 바삭한 튀김옷과 다양한 소스로 맛을 낸 치킨은 맥주와 함께하면 더욱 좋습니다.\n2. **피자**: 치즈의 풍부한 맛과 다양한 토핑이 어우러진 피자는 여럿이 함께 먹기에도 좋은 야식입니다.\n3. **라면**: 간편하게 준비할 수 있으면서도 매콤하고 진한 국물이 일품인 라면은 밤에 특히 더 맛있습니다.\n4. **족발 또는 보쌈**: 고기의 쫄깃한 식감과 함께, 상추와 깻잎에 싸 먹는 재미가 있는 족발이나 보쌈도 좋은 선택입니다.\n5. **김밥 또는 분식**: 간단하게 먹을 수 있는 김밥이나 떡볶이, 순대 같은 분식류도 야식으로 인기가 많습니다.\n\n계절, 개인의 취향, 그리고 현재 건강 상태를 고려하여 선택하는 것이 좋습니다. 가볍게 먹고 싶다면 김밥이나 샐러드를, 좀 더 든든하게 먹고 싶다면 치킨이나 피자를 추천합니다.'}

### Agent 구성

In [13]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Pinecone VectorDB',
        func=qa.invoke,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        ) # 해당 vectorDB를 사용할 상황에 대한 정의(매우 중요)
    )
]

In [14]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3, # query intent 파악 횟수
    early_stopping_method='generate',
    memory=conversational_memory
)

C:\Users\sckim\.conda\envs\vectordb\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [15]:
# 정상적인 질문해보기
agent(query)

C:\Users\sckim\.conda\envs\vectordb\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
```json
{
    "action": "Pinecone VectorDB",
    "action_input": "지미 카터"
}
```
Observation: {'query': '지미 카터', 'result': '제임스 얼 카터 주니어(1924년 10월 1일 ~ )는 민주당 출신으로 미국의 39대 대통령(1977년 ~ 1981년)을 역임했습니다. 그는 조지아주 섬터 카운티 플레인스 마을에서 태어났으며, 조지아 공과대학교를 졸업한 후 해군에 들어가 전함, 원자력, 잠수함의 승무원으로 일했습니다. 1953년에는 미국 해군 대위로 예편했고, 이후 땅콩과 면화 등을 재배하여 많은 돈을 벌었습니다. 그의 별명은 "땅콩 농부"(Peanut Farmer)로 알려져 있습니다.\n\n퇴임 후에는 카터 재단을 설립하여 민주주의 실현을 위한 제3세계의 선거 감시 활동, 기니 벌레에 의한 드라쿤쿠르스 질병 방재, 미국의 빈곤층 지원 활동, 사랑의 집짓기 운동, 국제 분쟁 중재 등 다양한 활동을 펼쳤습니다. 카터는 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁 등에서 군사적 행동을 최우선으로 선택하는 것에 대해 깊은 유감을 표하며, 미국의 군사적 활동에 강한 반대 입장을 보여왔습니다.'}
Thought:```json
{
    "action": "Final Answer",
    "action_input": "제임스 얼 카터 주니어는 1924년 10월 1일에 태어나 민주당 출신으로 미국의 39대 대통령(1977년 ~ 1981년)을 역임한 인물입니다. 그는 조지아주에서 태어나 조지아 공과대학교를 졸업한 후 해군에서 전함, 원자력, 잠수함 승무원으로 일했습니다. 해군 대위로 예편한 후에는 땅콩과 면화 재배로 성공을 거두었으며, '땅콩 농부'라는 별명으로도 알려져 있습니다. 퇴임 후에는 카터 재단을 설립하여 민주주의 실현을 위한 활동, 기니 벌레 질병 방재, 빈곤층 지원, 사랑의 집짓기 운동, 국제 분쟁 중재 등

{'input': '지미 카터가 누구야?',
 'chat_history': [],
 'output': "제임스 얼 카터 주니어는 1924년 10월 1일에 태어나 민주당 출신으로 미국의 39대 대통령(1977년 ~ 1981년)을 역임한 인물입니다. 그는 조지아주에서 태어나 조지아 공과대학교를 졸업한 후 해군에서 전함, 원자력, 잠수함 승무원으로 일했습니다. 해군 대위로 예편한 후에는 땅콩과 면화 재배로 성공을 거두었으며, '땅콩 농부'라는 별명으로도 알려져 있습니다. 퇴임 후에는 카터 재단을 설립하여 민주주의 실현을 위한 활동, 기니 벌레 질병 방재, 빈곤층 지원, 사랑의 집짓기 운동, 국제 분쟁 중재 등 다양한 사회 공헌 활동을 펼쳤습니다. 카터는 미국의 군사적 행동에 대해 강한 반대 입장을 보여왔습니다."}

In [16]:
# 다시 쓸데없는 질문 해보기
agent("오늘 야식 메뉴 추천좀 해줘")

# Agent 구성 의도에 맞게 vectorDB를 사용하지 않고 답변



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "야식으로는 피자, 치킨, 라면, 김밥, 떡볶이와 같은 다양한 메뉴가 인기가 많습니다. 기분이나 상황에 따라 가벼운 스낵에서부터 든든한 식사까지 선택할 수 있어요. 오늘 같은 날에는 따뜻한 국물이 있는 라면이나, 매콤달콤한 떡볶이가 좋을 것 같네요!"
}
```

> Finished chain.


{'input': '오늘 야식 메뉴 추천좀 해줘',
 'chat_history': [HumanMessage(content='지미 카터가 누구야?'),
  AIMessage(content="제임스 얼 카터 주니어는 1924년 10월 1일에 태어나 민주당 출신으로 미국의 39대 대통령(1977년 ~ 1981년)을 역임한 인물입니다. 그는 조지아주에서 태어나 조지아 공과대학교를 졸업한 후 해군에서 전함, 원자력, 잠수함 승무원으로 일했습니다. 해군 대위로 예편한 후에는 땅콩과 면화 재배로 성공을 거두었으며, '땅콩 농부'라는 별명으로도 알려져 있습니다. 퇴임 후에는 카터 재단을 설립하여 민주주의 실현을 위한 활동, 기니 벌레 질병 방재, 빈곤층 지원, 사랑의 집짓기 운동, 국제 분쟁 중재 등 다양한 사회 공헌 활동을 펼쳤습니다. 카터는 미국의 군사적 행동에 대해 강한 반대 입장을 보여왔습니다.")],
 'output': '야식으로는 피자, 치킨, 라면, 김밥, 떡볶이와 같은 다양한 메뉴가 인기가 많습니다. 기분이나 상황에 따라 가벼운 스낵에서부터 든든한 식사까지 선택할 수 있어요. 오늘 같은 날에는 따뜻한 국물이 있는 라면이나, 매콤달콤한 떡볶이가 좋을 것 같네요!'}